# Compare results for different years

In [ ]:
from pathlib import Path
import pandas as pd
from notebook_utils import get_formatted_score_df
from asr_eval import utils

mean_score_file = Path("../data/output/2024/mean_scores_year.csv")

if not mean_score_file.exists():
    p_previous = Path("../data/output/2023")
    p_current = Path("../data/output/2024")

    df = pd.concat(
        [get_formatted_score_df(p_previous), get_formatted_score_df(p_current)],
        ignore_index=True,
    )

    mean_score_df = utils.calculate_mean_scores(df, "year")
    mean_score_df.to_csv(mean_score_file, index=False)
else:
    mean_score_df = pd.read_csv(mean_score_file, dtype={"year": str})

In [ ]:
# remove chirp from 2025 data (only looking at chirp_2)
mean_score_df = mean_score_df[
    ~((mean_score_df.modell == "chirp") & (mean_score_df.year == "2025"))
]
assert (
    len(mean_score_df[mean_score_df.modell == "chirp"]) == 1
    and len(mean_score_df[mean_score_df.modell == "chirp_2"]) == 1
)

# rename chirp_2 to chirp
mean_score_df["modell"] = mean_score_df.modell.apply(
    lambda x: x if "chirp" not in x else "chirp"
)
assert (
    len(mean_score_df[mean_score_df.modell == "chirp"]) == 2
    and len(mean_score_df[mean_score_df.modell == "chirp_2"]) == 0
)

In [ ]:
# Rename all wav2vec2 models to wav2vec2
def rename_wav2vec(model_name: str):
    if "nb-wav2vec" in model_name:
        return "nb-wav2vec2"
    return model_name


mean_score_df["modell"] = mean_score_df.modell.apply(rename_wav2vec)

In [ ]:
models_to_keep = [
    "nb-whisper-large",
    "nb-whisper-large-verbatim",
    "nb-wav2vec2",
    "openai-whisper-large",
    "openai-whisper-large-v3-turbo",
    "gcloud",
    "chirp",
    "azure",
    "facebook_mms-1b-all",
]

mean_score_df = mean_score_df[mean_score_df.modell.isin(models_to_keep)]
mean_score_df

# Plott scores per år (språk hver for seg)

In [ ]:
from notebook_utils import make_plot

imagedir = Path("images")
imagedir.mkdir(exist_ok=True)

lang_map = {"nob": "bokmål", "nno": "nynorsk"}
metric_range_map = {
    "CER": range(10, 50, 10),
    "WER": range(10, 80, 10),
    "semantic distance (sBERT)": [i / 100 for i in range(5, 45, 5)],
    "semantic distance": [i / 100 for i in range(5, 45, 5)],
    "aligned semantic distance": [i / 10 for i in range(1, 8, 1)],
}

for metric in [
    "CER",
    "WER",
    "semantic distance (sBERT)",
    "semantic distance",
    "aligned semantic distance",
]:
    for language in ["nno", "nob"]:
        make_plot(
            df=mean_score_df,
            plot_type="barchart",
            feature="year",
            metric=metric,
            language=language,
            save_to_dir=imagedir,
            score_display_range=metric_range_map[metric],
        )

In [ ]:
# Annen plottemåte

import plotly.express as px
from plotly.colors import qualitative

imagedir = Path("images")
imagedir.mkdir(exist_ok=True)

lang_map = {"nob": "bokmål", "nno": "nynorsk"}

for metric in [
    "CER",
    "WER",
    "semantic distance (sBERT)",
    "semantic distance",
    "aligned semantic distance",
]:
    for language in ["nno", "nob"]:
        fig = px.bar(
            mean_score_df[mean_score_df.språk == language].sort_values(
                by=["year", "modell", metric], ascending=True
            ),
            x="modell",
            y=metric,
            color="year",
            color_discrete_map={
                "2024": qualitative.Pastel2[0],
                "2025": qualitative.Dark2[0],
            },
            barmode="group",
            labels={
                "modell": "Modell",
                metric: "Gjennomsnittscore",
                "språk": "Språk",
                "year": "År",
            },
            title=f"Gjennomsnittlig {metric} for {lang_map[language]}",
        )
        fig.write_image(
            imagedir / f"barchart_year_{'-'.join(metric.split())}_{language}.png"
        )

# Plot scores og språk per år 

In [ ]:
import plotly.express as px

# Define the color mapping for (language, year) combinations
color_map = {
    ("nno", "2025"): "#64aedc",
    ("nno", "2024"): "#c8e3f3",
    ("nob", "2025"): "#9ed67f",
    ("nob", "2024"): "#def1d3",
}

# Define custom labels for the legend
custom_labels = {
    ("nno", "2025"): "nynorsk 2025",
    ("nno", "2024"): "nynorsk 2024",
    ("nob", "2025"): "bokmål 2025",
    ("nob", "2024"): "bokmål 2024",
}

mean_score_df["custom_label"] = mean_score_df[["språk", "year"]].apply(
    lambda x: custom_labels[tuple(x)], axis=1
)

metric = "CER"

for metric in [
    "CER",
    "WER",
    "semantic distance (sBERT)",
    "semantic distance",
    "aligned semantic distance",
]:
    # Create the grouped bar chart
    fig = px.bar(
        mean_score_df,
        x="modell",
        y=metric,  # Replace with the actual metric column name
        color="custom_label",  # Use the custom label column for the legend
        color_discrete_map={v: color_map[k] for k, v in custom_labels.items()},
        barmode="group",
        labels={
            "modell": "",
            "metric_column": "Metric Score",  # Replace with the actual metric column name
            "custom_label": "",
        },
        # title=f"Gjennomsnittlig {metric} fordelt på modell språk og år",
    )

    # Update layout to set background, grid lines, and add a black border
    fig.update_layout(
        plot_bgcolor="white",  # Set plot background to white
        paper_bgcolor="white",  # Set paper background to white
        xaxis=dict(showgrid=False),  # Remove vertical grid lines
        yaxis=dict(
            gridcolor="lightgrey",  # Set horizontal grid lines to grey
        ),
        shapes=[
            dict(
                type="rect",
                xref="paper",
                yref="paper",
                x0=0,
                y0=0,
                x1=1,
                y1=1,
                line=dict(color="black", width=0.5),  # Add a black border
            )
        ],
    )

    # Show the figure
    fig.show()
    fig.write_image(imagedir / f"barchart_{'-'.join(metric.split())}.png")